In [2]:
import numpy as np
import pandas as pd 
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [3]:
# Loading data files.
app_data = pd.read_csv('D:\Моя учёба\ДИПЛОМ\моя нейросеть\data_train.csv',sep = ";")
app_data = app_data.drop(app_data.columns[0], axis=1)
app_data.head()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,person_home_ownership_0,...,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,loan_grade_A,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G
0,22,59000,123.0,35000,16.02,1,0.59,1,3,0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,21,9600,5.0,1000,11.14,0,0.10,0,2,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,25,9600,1.0,5500,12.87,1,0.57,0,3,0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,23,65500,4.0,35000,15.23,1,0.53,0,2,0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,24,54400,8.0,35000,14.27,1,0.55,1,4,0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [4]:
app_data['loan_int_rate'].replace('', np.nan, inplace=True)
app_data['person_emp_length'].replace('', np.nan, inplace=True)
app_data.dropna(subset=['loan_int_rate'], inplace=True)
app_data.dropna(subset=['person_emp_length'], inplace=True)

In [5]:
app_data = pd.get_dummies(app_data)

In [6]:
print("Shape of application data", app_data.shape)

Shape of application data (28638, 25)


In [7]:
y = app_data['loan_status']
X = app_data.drop(['loan_status'], axis=1)

In [8]:
imputer = SimpleImputer(strategy='median')
imputer.fit(X)
X = imputer.transform(X)

In [9]:
scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(X)
X = scaler.transform(X)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

In [11]:
# Логистическая регрессия
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()
lr_model.fit(X_train,y_train)

LogisticRegression()

In [12]:
y_pred = lr_model.predict(X_test)

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
acc = accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
print("Точность логистической регрессии - ", acc)
print("MSE логистической регрессии - ", mse)

Точность логистической регрессии -  0.8659649822916147
MSE логистической регрессии -  0.1340350177083853


In [14]:
# RandomForest Classifier
from sklearn.ensemble import RandomForestClassifier
random_clf = RandomForestClassifier(n_estimators=50)

random_clf.fit(X_train, y_train)
y_pred_r = random_clf.predict(X_test)

# точность метода
y_test_pred = random_clf.predict(X_test)
acc_rf = accuracy_score(y_test, y_test_pred)
mse_rf = mean_squared_error(y_test, y_pred_r, squared=True)
print("Точность метода RandomForest - ", acc_rf)
print("MSE метода RandomForest - ", mse_rf)

Точность метода RandomForest -  0.9270713822517085
MSE метода RandomForest -  0.07292861774829151


In [15]:
# Нейросеть
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

In [16]:
app_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28638 entries, 0 to 32580
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   person_age                     28638 non-null  int64  
 1   person_income                  28638 non-null  int64  
 2   person_emp_length              28638 non-null  float64
 3   loan_amnt                      28638 non-null  int64  
 4   loan_int_rate                  28638 non-null  float64
 5   loan_status                    28638 non-null  int64  
 6   loan_percent_income            28638 non-null  float64
 7   cb_person_default_on_file      28638 non-null  int64  
 8   cb_person_cred_hist_length     28638 non-null  int64  
 9   person_home_ownership_0        28638 non-null  int64  
 10  person_home_ownership_1        28638 non-null  int64  
 11  person_home_ownership_2        28638 non-null  int64  
 12  loan_intent_DEBTCONSOLIDATION  28638 non-null 

In [17]:
# Задание структуры нейросети
model = Sequential()
model.add(Dense(24,input_dim=24, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(42, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                600       
_________________________________________________________________
dropout (Dropout)            (None, 24)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 42)                1050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 42)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 43        
Total params: 1,693
Trainable params: 1,693
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','mse'])

In [20]:
model.fit(X_train, y_train, epochs=100, batch_size=500,
          verbose=0)

In [21]:
evl = model.evaluate(X_test, y_test)

627/627 [==============================] - 1s 794us/step - loss: 0.2759 - accuracy: 0.9017 - mse: 0.0792


In [22]:
print("Точность данной модели нейросети - ", evl[1])

Точность данной модели нейросети -  0.9017309546470642
